In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as itt
import statsmodels as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

C:\Users\korea\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#define function
def parameter_search_sarima(train, test, params):
    itmnbrlists=[1430040]
    optimal_params={}
    results={}
    
    for itmnbr in itmnbrlists:
        bev_itm=train.iloc[train.index[train.item_nbr==itmnbr]]
#         print(bev_itm)
        dates=bev_itm.date
#         print(dates)
        unitsales=bev_itm.unit_sales
        ts_log=pd.concat([dates, np.log(unitsales)], axis=1)
        ts_log=ts_log.set_index('date')

        log_unit_sales=ts_log.unit_sales
        
        bev_itmt=test.iloc[test.index[test.item_nbr==itmnbr]]
        datest=bev_itmt.date
        unitsalest=bev_itmt.unit_sales
        tst_log=pd.concat([datest, np.log(unitsalest)], axis=1)
        tst_log=tst_log.set_index('date')
        log_unit_salest=tst_log.unit_sales
        
        mse_train=[]
        mse_test=[]
        aics=[]
        
        new_params=[]
        
        for i in range(len(params)):
        
            param=params[i]
            
            if (param[0] == 0 and param[1] != 0 and param[2]==0) or (param[3]==0 and param[4]!=0 and param[5]==0):
                print('{} is skipped'.format(param))
                continue
            
            order1=param[:3]
            order2=param[3:]+(7,)

            print('{} : modelling...:{}'.format(itmnbr, param))
            try:
                model=SARIMAX(ts_log, order=order1,seasonal_order=order2, trend='c',enforce_invertibility=False)
            except:

                print('{} raised Value Error. Go to next param'.format(param))                
                continue

            try:
                result=model.fit()
                aic=result.aic
            except:
                print('{} raised LinAlgError. Go to next param'.format(param))
                continue



            preds_log=result.predict(start=1, end=len(ts_log)+len(tst_log))
            dates_test=pd.date_range(start=dates.values[0], periods=len(preds_log))

            try:
                mse_tr=mean_squared_error(unitsales, np.exp(preds_log, dtype=np.float64)[:len(ts_log)])
                mse_tt=mean_squared_error(unitsalest, np.exp(preds_log, dtype=np.float64)[len(ts_log):])
            except:
                print('{} raised overflow error. Go to next param'.format(param))
            
            mse_train.append(mse_tr)
            mse_test.append(mse_tt)
            aics.append(aic)
            new_params.append(param)

        ind_min_tr=np.argmin(mse_train)
        ind_min_tt=np.argmin(mse_test)
        ind_min_aic=np.argmin(aics)

        mse_atr=mse_train[ind_min_tr]
        opt_atr=new_params[ind_min_tr]
        opt_att=new_params[ind_min_tt]
        mse_att=mse_test[ind_min_tt]
        aic_min=aics[ind_min_aic]
        opt_aic=new_params[ind_min_aic]

        results[itmnbr]=[mse_train , mse_test, new_params, aics]
        optimal_params[itmnbr]={'mse':[opt_atr, mse_atr], 'mse2':[opt_att, mse_att], 'aic':[opt_aic, aic_min]}

        print('{0} -> mse_param_train:{1}, mse_train:{2}'.format(itmnbr, opt_atr, mse_atr))
        print('{0} -> mse_param_test:{1}, mse_test:{2}'.format(itmnbr, opt_att, mse_att))
        print('{0} -> aic_param:{1}, aic_min:{2}'.format(itmnbr, opt_aic, aic_min))
        
    return results, optimal_params
        
        

            

In [3]:
train=pd.read_csv('../BEV_train.csv', parse_dates=[0], infer_datetime_format=True)
train=train.drop('family', axis=1)
train=train[['date','item_nbr','unit_sales']]
test=pd.read_csv('../BEV_test.csv', parse_dates=[0], infer_datetime_format=True)
test=test.drop('family', axis=1)
test=test[['date','item_nbr','unit_sales']]
params=list(itt.product([0,1,2,3],[0,1,2,3],[0,1,2,3],[0,1,2,3],[0,1,2,3],[0,1,2,3],[7]))

In [4]:
results, optimal_params=parameter_search_sarima(train, test, params)

1430040 : modelling...:(0, 0, 0, 0, 0, 0, 7)
(0, 0, 0, 0, 0, 0, 7) raised Value Error. Go to next param
1430040 : modelling...:(0, 0, 0, 0, 0, 1, 7)
1430040 : modelling...:(0, 0, 0, 0, 0, 2, 7)
1430040 : modelling...:(0, 0, 0, 0, 0, 3, 7)
(0, 0, 0, 0, 1, 0, 7) is skipped
1430040 : modelling...:(0, 0, 0, 0, 1, 1, 7)
1430040 : modelling...:(0, 0, 0, 0, 1, 2, 7)
1430040 : modelling...:(0, 0, 0, 0, 1, 3, 7)
(0, 0, 0, 0, 2, 0, 7) is skipped
1430040 : modelling...:(0, 0, 0, 0, 2, 1, 7)
1430040 : modelling...:(0, 0, 0, 0, 2, 2, 7)
1430040 : modelling...:(0, 0, 0, 0, 2, 3, 7)
(0, 0, 0, 0, 3, 0, 7) is skipped
1430040 : modelling...:(0, 0, 0, 0, 3, 1, 7)
1430040 : modelling...:(0, 0, 0, 0, 3, 2, 7)
1430040 : modelling...:(0, 0, 0, 0, 3, 3, 7)
1430040 : modelling...:(0, 0, 0, 1, 0, 0, 7)
1430040 : modelling...:(0, 0, 0, 1, 0, 1, 7)


C:\Users\korea\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


1430040 : modelling...:(0, 0, 0, 1, 0, 2, 7)


C:\Users\korea\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


1430040 : modelling...:(0, 0, 0, 1, 0, 3, 7)


...

1430040 : modelling...:(3, 3, 3, 3, 3, 0, 7)
(3, 3, 3, 3, 3, 0, 7) raised overflow error. Go to next param
1430040 : modelling...:(3, 3, 3, 3, 3, 1, 7)
(3, 3, 3, 3, 3, 1, 7) raised overflow error. Go to next param
1430040 : modelling...:(3, 3, 3, 3, 3, 2, 7)


C:\Users\korea\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


(3, 3, 3, 3, 3, 2, 7) raised overflow error. Go to next param
1430040 : modelling...:(3, 3, 3, 3, 3, 3, 7)


C:\Users\korea\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


(3, 3, 3, 3, 3, 3, 7) raised overflow error. Go to next param
1430040 -> mse_param_train:(1, 1, 0, 0, 0, 0, 7), mse_train:4127.131899181152
1430040 -> mse_param_test:(2, 1, 1, 0, 1, 2, 7), mse_test:37280.74193426736
1430040 -> aic_param:(1, 0, 0, 1, 0, 0, 7), aic_min:1221.9610211599218


In [29]:
results

{364738: [[855.10081733708751,
   829.8930483427805,
   844.14519711721005,
   879.55085715735697,
   1573.1742555336994,
   1397.5433260080304,
   811.22057071376764,
   803.14731219729867,
   802.92014661563132,
   891.54921596840074,
   849.4664347280376,
   835.98875432420584,
   14583.486830623851,
   1480.3616031993229,
   1355.8350293934025,
   802.78866317353436,
   802.99547172677148,
   796.66980696752603,
   879.24781592120121,
   837.50640736210948,
   835.87120226908314,
   7047.9031110187634,
   1464.8646028098312,
   1346.4840187588366,
   734.83920132824539,
   709.38178726307206,
   698.98942499753377,
   730.26117670633823,
   744.65733938238895,
   1461.9243393646275,
   1266.191543052553,
   693.45359895297872,
   681.9499800020244,
   680.86111831431856,
   759.66831312318857,
   736.51211610547784,
   720.23426021000591,
   9557.5896118650289,
   1345.0058236362831,
   1257.040212294904,
   682.96459619931738,
   681.21331281760922,
   678.50405990582726,
   743.6

In [31]:
optimal_params

{364738: {'aic': [(1, 0, 2, 2, 0, 0), 1534.1832153339287],
  'mse': [(0, 2, 1, 0, 0, 0), 10.244979574426358],
  'mse2': [(1, 1, 0, 1, 0, 1), 304.06713021524428]},
 464374: {'aic': [(1, 1, 1, 1, 0, 0), 1281.1384170760414],
  'mse': [(0, 2, 1, 0, 0, 0), 0.92309601968426347],
  'mse2': [(1, 2, 2, 2, 0, 0), 431.53188132480079]},
 716250: {'aic': [(1, 0, 0, 2, 0, 0), 1421.285686078274],
  'mse': [(0, 2, 1, 0, 0, 0), 0.40137787119163532],
  'mse2': [(0, 0, 0, 1, 1, 1), 10.128869310708616]},
 1047679: {'aic': [(1, 0, 2, 1, 0, 1), 1518.4853560947208],
  'mse': [(0, 2, 1, 0, 0, 0), 14.292197321320355],
  'mse2': [(0, 2, 1, 2, 0, 1), 569.69964526258593]}}

In [32]:
import pickle
with open('../results/optimal_params4.pickle', 'wb') as f:
    pickle.dump(optimal_params, f)

In [33]:
import pickle
with open('../results/total_results4.pickle', 'wb') as f:
    pickle.dump(results, f)